# Ensemble

## Load data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib 
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
from collections import defaultdict
import datetime
from datetime import datetime
import pickle
import math
import os

from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import Lasso,Ridge
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone


pd.set_option('display.max_columns', None) # no truncate columns

In [2]:
# Environment settings
data_path_out = 'Data/output/'
    
# Deserialize previously saved data from "data-visualization"
with open(data_path_out + 'train_pp.obj', 'rb') as file:
    all_train = pickle.load(file)
all_train.columns

Index(['StoreID', 'Date', 'IsHoliday', 'IsOpen', 'HasPromotions',
       'NearestCompetitor', 'Region_AreaKM2', 'Region_GDP',
       'Region_PopulationK', 'CloudCover', 'Max_Dew_PointC', 'Max_Humidity',
       'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm',
       'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
       'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
       'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM',
       'Precipitationmm', 'WindDirDegrees', 'Events_Hail', 'Hol_and_open',
       'Region_PD', 'week_of_month', 'year', 'quarter', 'month',
       'day_of_month', 'day_of_week', 'day_of_year', 'WeekOfYear',
       'days_in_month', 'Region_0', 'Region_1', 'Region_10', 'Region_2',
       'Region_3', 'Region_4', 'Region_5', 'Region_7', 'Region_9', 'Region_6',
       'Region_8', 'AssortmentType_General',
       'AssortmentType_With Non-F

In [3]:
all_train_orig = all_train.copy()

### Drop now useless variables

In [4]:
all_train = all_train.drop(labels = ['Region'],axis=1)

## Ensemble model

### Averaging model function

In [5]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

### Stacked Model function

In [6]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

## General Model

In [ ]:
all_dates = pd.DatetimeIndex(all_train['Date'])
month_dict = { d.strftime("%B%Y") :datetime.strptime(d.strftime('%m-%Y'),'%m-%Y') for d in all_dates}

In [ ]:
def monthly_training(training_interval=['03-2017','02-2018'],
                     verbose=False,
                     regressor=RandomForestRegressor(n_estimators=50,n_jobs=-1)):
    
    init_date=datetime.strptime(training_interval[0],'%m-%Y')
    end_date =datetime.strptime(training_interval[1],'%m-%Y')
    
    month_scores = []
    y_test_list = []
    predicted_list= []
    predicted_df_list = []
    training_months = []
    training_df_list = []
    
    for month_name, date in month_dict.items():

        if (date >= init_date) & (date<= end_date) :
                if verbose== True:
                    print("Doing month",month_name)
                    
                #just numerical month names
                training_months.append(month_name)
                
                #TEST TRAIN GENERATION
                #--------------------
                mask = ((all_train['month']==date.month) & (all_train['year']==date.year))
                test = all_train[mask]
                train = all_train[~mask]
                
                
                columns_to_drop=['Date','CloudCover', 'Max_Dew_PointC', 'Max_Humidity',
                                 'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm',
                                 'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity',
                                 'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
                                 'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
                                 'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM',
                                 'Precipitationmm', 'WindDirDegrees']
                

        
                train = train.drop(columns_to_drop,axis=1)
                test = test.drop(columns_to_drop,axis=1)
                
                
                y_train = train.NumberOfSales
                X_train = train.drop('NumberOfSales',axis = 1)

                y_test = test.NumberOfSales
                X_test = test.drop('NumberOfSales',axis = 1)
                #-------------------------------

                
                #SCALING
                #--------------------------------
                columns_to_scale = ['NearestCompetitor', 
                                    'Region_AreaKM2',
                                    'Region_GDP',
                                    'Region_PopulationK',
                                    'Region_PD']
    
                scaler = RobustScaler()
                X_train[columns_to_scale]=scaler.fit_transform(X_train[columns_to_scale])
                X_test[columns_to_scale]=scaler.transform(X_test[columns_to_scale])
                reg =  regressor.fit(X_train,y_train)
                #--------------------------------

                
                #FEATURE IMPORTANCE
               #--------------------
                feat_labels = train.columns[0:]
                importances = reg.feature_importances_
                indices = np.argsort(importances)[::-1]
                for f in range(0,10):
                    print("%2d %-*s %f" %(f+1,30,feat_labels[indices[f]],importances[indices[f]]))
               #--------------------


                
                #FITTING AND SCORING
                #--------------------
                current_score = reg.score(X_test,y_test)
                current_prediction = reg.predict(X_test)
                
                month_scores.append(current_score)
                predicted_list.append(current_prediction)
                y_test_list.append(y_test)
                #--------------------

                
                if verbose == True:
                    print("-Month {} has shape {}\n\t".format(month_name,test.shape))
                    print("-Score {:.3f}".format(current_score))
                    
                    
                
                # SAVE DATA FOR PLOTTING/PRINTING
                #--------------------
                real_df = all_train.copy()
                month_test_df = all_train[mask]
                month_all_train= all_train[~mask]

                month_test_df= month_test_df.drop('NumberOfSales',axis=1)
                prediction_=pd.Series(current_prediction)
                month_test_df['NumberOfSales']=prediction_.values.astype(int)
                predicted_df = pd.concat([month_test_df,month_all_train]).reset_index()
                predicted_df= predicted_df[list(all_train.columns.values)]
                
                predicted_df_list.append(predicted_df)
                training_df_list.append(train)
                #--------------------

                
    return {
        'Scores' : month_scores,
        'Real' : y_test_list,
        'Predictions' : predicted_list,
        'Training_dates': training_months,
        'Training_df' : training_df_list,
        'Predicted_df' : predicted_df_list
    }

In [ ]:
# reg = RandomForestRegressor(n_estimators=500,n_jobs=-1,verbose=True)

model = ExtraTreesRegressor(n_estimators=50,n_jobs=-1,verbose=2)

prediction_result = monthly_training(verbose=True,
                                     training_interval=['12-2017','01-2018'],
                                     regressor = model)

In [7]:
all_train = all_train.drop('Date',axis=1)

## Cross-validation

In [16]:
columns_to_drop=['CloudCover', 'Max_Dew_PointC', 'Max_Humidity',
                                 'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm',
                                 'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity',
                                 'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
                                 'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
                                 'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM',
                                 'Precipitationmm','WindDirDegrees']

X = all_train.drop('NumberOfSales',axis=1)
y = all_train.NumberOfSales
X = X.drop(columns_to_drop,axis=1)
#SCALING
#--------------------------------
columns_to_scale = ['NearestCompetitor',
                    'Region_AreaKM2',
                    'Region_GDP',
                    'Region_PopulationK',
                    'Region_PD']

scaler = RobustScaler()
X[columns_to_scale]=scaler.fit_transform(X[columns_to_scale])

model = ExtraTreesRegressor()

kf = KFold(5, shuffle=True, random_state=42)
r2_cv_xt= cross_val_score(model, X, y, scoring="r2", cv = kf,verbose=2,n_jobs=-1)


print("The 10 -fold crossvalidation R2 of XT is {:.5f} +/- {:.3f}".format(r2_cv_xt.mean(),r2_cv_xt.std()))

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total= 1.8min
[CV] ................................................. , total= 1.8min
[CV]  ................................................................
[CV] ................................................. , total= 1.8min
[CV] ................................................. , total= 1.8min
[CV] ................................................. , total=  45.4s
The 10 -fold crossvalidation R2 of XT is 0.92924 +/- 0.001


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.6min finished


In [ ]:
# df_results = pd.DataFrame.from_dict(results).T
# df_results_mean = df_results.mean(axis=1)
# df_results_mean[df_results_mean < 0.8]

## Lanzi Error

### Fit model and make predictions

In [17]:
# train by month
def split_dataset_bymonth(test_year, test_months, train_set):
    test_mask = (train.year == test_year) & train.month.isin(test_months)
    
    # define the train set
    train_dataset = train[~test_mask]
    x_train = train_dataset.drop('NumberOfSales', axis=1)
    y_train = train_dataset.NumberOfSales
    
    # define the test set
    test_dataset = train[test_mask]
    x_test = test_dataset.drop('NumberOfSales', axis=1)
    y_test = test_dataset.NumberOfSales
    
    return (x_train, y_train, x_test, y_test)

In [ ]:




scores = {}
scores_mae= {}
predictions = defaultdict(dict)
store_pred = {}
shopping_center_ids = [1129,1267,1280,1307,1330,1339,1357,1387,1676]
region2_stores = all_train[all_train['Region_2'] ==1]['StoreID'].unique()
ids = all_train.StoreID.unique()

for storeid in ids:
    
    model1= Lasso(alpha=20)
    model2 =XGBRegressor(n_estimators=100)
    model3 = GradientBoostingRegressor(n_estimators=50, max_depth=5,learning_rate=0.07,
                                      loss='huber',random_state =5)
    
    model4 = ExtraTreesRegressor(n_estimators=50)

    
    model = AveragingModels(models = (model1,model2,model3,model4)) 

    # split the dataset
    train = all_train[all_train.StoreID == storeid]
    
    x_train, y_train, x_test, y_test =\
    split_dataset_bymonth(2017,[3,4], train)
    
    

    
    
    # train the model with the training set
    model.fit(x_train, y_train)
              
    # scoring
    scores[storeid] = r2_score(y_test, model.predict(x_test))
    scores_mae[storeid]= mean_absolute_error(y_test, model.predict(x_test))
    
    print('store r2 {} -> {:.4f}'.format(storeid, scores[storeid]))
#     print('store mae {} -> {:.2f}'.format(storeid, scores_mae[storeid]))
    
    store_pred[storeid] = scores[storeid]
    # predict the test set with the trained model
    for month in x_test.month.unique():
        # get daily predictions for each month in the test set
        month_prediction =model.predict(x_test[x_test.month == month]).astype("int")
        month_actual = y_test.loc[x_test[x_test.month == month].index].values
        
#         if scores[storeid] <0.6 :
#             ts_pred = pd.Series(month_prediction, index=x_test[x_test.month==month]['day_of_month']).\
#                 plot(figsize=(20,5), title='Region 2', marker='o')
#             ts_act = pd.Series(month_actual, index=x_test[x_test.month==month]['day_of_month']).\
#                 plot(figsize=(20,5), title='Region 2', marker='x')
#             plt.show()
        
        # store the monthly mean of the test set
        predictions[storeid][month] = {
            'predicted': np.sum(month_prediction),
            'actual': np.sum(month_actual)        
        }
#     predictions[storeid]['r2']=scores[storeid]
#     predictions[storeid]['mae']=scores_mae[storeid]


store r2 1000 -> 0.8211
store r2 1001 -> 0.8331
store r2 1002 -> 0.9616
store r2 1003 -> 0.9474
store r2 1004 -> 0.9498
store r2 1005 -> 0.9133
store r2 1006 -> 0.9270
store r2 1007 -> 0.9018
store r2 1008 -> 0.8683
store r2 1009 -> 0.9427
store r2 1010 -> 0.9122
store r2 1011 -> 0.9127
store r2 1012 -> 0.9684
store r2 1013 -> 0.9722
store r2 1014 -> 0.9230
store r2 1015 -> 0.9522
store r2 1016 -> 0.8988
store r2 1017 -> 0.9561
store r2 1018 -> 0.9344
store r2 1019 -> 0.9670
store r2 1020 -> 0.9141
store r2 1021 -> 0.9675
store r2 1022 -> 0.9336
store r2 1023 -> 0.9644
store r2 1024 -> 0.9530
store r2 1025 -> 0.8560
store r2 1026 -> 0.9619
store r2 1027 -> 0.9334
store r2 1028 -> 0.8847
store r2 1029 -> 0.9567
store r2 1030 -> 0.9258
store r2 1031 -> 0.8751
store r2 1032 -> 0.9667
store r2 1033 -> 0.9801
store r2 1034 -> 0.9035
store r2 1035 -> 0.9670
store r2 1036 -> 0.9329
store r2 1037 -> 0.8986
store r2 1038 -> 0.9249
store r2 1039 -> 0.9566
store r2 1040 -> 0.9575
store r2 1041 ->

/Users/yannickgiovanakis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


store r2 1057 -> 0.9450
store r2 1058 -> 0.9089
store r2 1059 -> 0.9782
store r2 1060 -> 0.9637
store r2 1061 -> 0.9687
store r2 1062 -> 0.9655
store r2 1063 -> 0.9702
store r2 1064 -> 0.9631
store r2 1065 -> 0.9210
store r2 1066 -> 0.9404
store r2 1067 -> 0.9331
store r2 1068 -> 0.7672
store r2 1069 -> 0.9126
store r2 1070 -> 0.9417
store r2 1071 -> 0.9479
store r2 1072 -> 0.9404
store r2 1073 -> 0.9532
store r2 1074 -> 0.9299
store r2 1075 -> 0.9214
store r2 1076 -> 0.9398
store r2 1077 -> 0.8410
store r2 1078 -> 0.9672
store r2 1079 -> 0.9284
store r2 1080 -> 0.8374
store r2 1081 -> 0.9539
store r2 1082 -> 0.9160
store r2 1083 -> 0.9049
store r2 1084 -> 0.9624
store r2 1085 -> 0.9578
store r2 1086 -> 0.9349
store r2 1087 -> 0.8467
store r2 1088 -> 0.9746
store r2 1089 -> 0.9403
store r2 1090 -> 0.7720
store r2 1091 -> 0.9688
store r2 1092 -> 0.9396
store r2 1093 -> 0.8917
store r2 1094 -> 0.9173
store r2 1095 -> 0.9474
store r2 1096 -> 0.9307
store r2 1097 -> 0.9432
store r2 1098 ->

/Users/yannickgiovanakis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


store r2 1280 -> 0.8579
store r2 1281 -> 0.9031
store r2 1282 -> 0.9240
store r2 1283 -> 0.9323
store r2 1284 -> 0.9122
store r2 1285 -> 0.8957
store r2 1286 -> 0.9694
store r2 1287 -> 0.9542
store r2 1288 -> 0.9388
store r2 1289 -> 0.9192
store r2 1290 -> 0.8918
store r2 1291 -> 0.8706
store r2 1292 -> 0.9292
store r2 1293 -> 0.9266
store r2 1294 -> 0.8610
store r2 1295 -> 0.9150
store r2 1296 -> 0.8968
store r2 1297 -> 0.9654
store r2 1298 -> 0.9468
store r2 1299 -> 0.9189
store r2 1300 -> 0.9661
store r2 1301 -> 0.9247
store r2 1302 -> 0.9623
store r2 1303 -> 0.8353
store r2 1304 -> 0.9589
store r2 1305 -> 0.8745
store r2 1306 -> 0.8833
store r2 1307 -> 0.6442
store r2 1308 -> 0.9765
store r2 1309 -> 0.9337
store r2 1310 -> 0.8469
store r2 1311 -> 0.9101
store r2 1312 -> 0.8655
store r2 1313 -> 0.9521
store r2 1314 -> 0.9295
store r2 1315 -> 0.8734
store r2 1316 -> 0.9553
store r2 1317 -> 0.9261
store r2 1318 -> 0.9322
store r2 1319 -> 0.9258
store r2 1320 -> 0.9786
store r2 1321 ->

/Users/yannickgiovanakis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


store r2 1346 -> 0.8705
store r2 1347 -> 0.9492
store r2 1348 -> 0.9105
store r2 1349 -> 0.8342
store r2 1350 -> 0.9555
store r2 1351 -> 0.8845
store r2 1352 -> 0.7956
store r2 1353 -> 0.9487
store r2 1354 -> 0.9391
store r2 1355 -> 0.9755
store r2 1356 -> 0.9309
store r2 1357 -> 0.8356
store r2 1358 -> 0.9701
store r2 1359 -> 0.8912
store r2 1360 -> 0.9296
store r2 1361 -> 0.9740
store r2 1362 -> 0.9350
store r2 1363 -> 0.9618
store r2 1364 -> 0.8571
store r2 1365 -> 0.9653
store r2 1366 -> 0.9543
store r2 1367 -> 0.7979
store r2 1368 -> 0.9249
store r2 1369 -> 0.9360
store r2 1370 -> 0.8124
store r2 1371 -> 0.9647
store r2 1372 -> 0.9478
store r2 1373 -> 0.9289
store r2 1374 -> 0.9087
store r2 1375 -> 0.9447
store r2 1376 -> 0.9640
store r2 1377 -> 0.9367
store r2 1378 -> 0.9197
store r2 1379 -> 0.9374
store r2 1380 -> 0.9168
store r2 1381 -> 0.9703
store r2 1382 -> 0.9324
store r2 1383 -> 0.9558
store r2 1384 -> 0.9412
store r2 1385 -> 0.9113
store r2 1386 -> 0.9736
store r2 1387 ->

### Compute Lanzi error

In [ ]:
# set of regions
R = sorted(all_train_orig.Region.unique().astype(int))
# set of predicted months
months = [key for key, value in predictions[1000].items()]
# set of stores by region
dict_store_byRegion = all_train_orig[['Region', 'StoreID']].drop_duplicates()\
.set_index('StoreID').groupby('Region').groups

def region_error(region, predictions):    
    num = 0
    den = 0
    for store in dict_store_byRegion[str(region)]:
        for month in months:
            predicted = predictions[store][month]['predicted']
            actual = predictions[store][month]['actual']
            
            num += abs(actual - predicted)
            den += actual
    
    return num/den
    
# total_error input:
#
# region_errors = [0.3, 0.5, ... ]

def total_error(region_errors):
#     print(region_errors)
    return sum(region_errors)/len(region_errors)

def lanzi_error(predictions):
    region_errors = []
    for r in R:
        region_errors.append(region_error(r, predictions))
    
    return total_error(region_errors)

In [ ]:
print('Lanzi error: {}'.format(lanzi_error(predictions)))